#PLOTEO DE RGB VAPOR DE AGUA , RGB DE MASA DE AIRE + VORTICIDAD

In [ ]:
!pip install cartopy
!pip install netCDF4
!pip install xarray
!pip install GOES
!pip install custom_color_palette

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from netCDF4 import Dataset
import xarray as xr
import custom_color_palette as ccp
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter

In [ ]:
# =========================================================
# 2. CONFIGURACIÓN DE RUTAS
# =========================================================
ruta_base = "/content/drive/MyDrive/GOES19_ATACAMA_20250624_20250630/"
era5_file = "/content/drive/MyDrive/DATOS/ERA5.nc"
# =========================================================
# CARPETA DE SALIDA DE GRÁFICOS
# =========================================================
output_dir = "/content/drive/MyDrive/FIGURAS_VAPOR_VORT500"
os.makedirs(output_dir, exist_ok=True)

domain = [-100, -40, -70, 10] # [Oeste, Este, Sur, Norte]

targets = [
    "20250624_12",
    "20250625_12",
    "20250626_12",
    "20250627_12",
    "20250628_12",
    "20250629_12",
    "20250630_12"
]


In [ ]:
import xarray as xr
ds = xr.open_dataset(era5_file)
ds


In [ ]:
# ============================
# FECHA ERA5
# ============================
fecha = f"{target[:4]}-{target[4:6]}-{target[6:8]}T{target[9:11]}:00"

era500 = era.sel(
    pressure_level=500,
    valid_time=fecha,
    method="nearest"
)


In [ ]:
# =========================================================
# 3. LIBRERIAS NECESARIOS
# =========================================================
import os
import gc
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
from scipy.ndimage import gaussian_filter
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter
import GOES




In [ ]:
# Ajuste de longitudes ERA5
if ds_era.longitude.max() > 180:
    ds_era['longitude'] = (ds_era.longitude + 180) % 360 - 180
    ds_era = ds_era.sortby('longitude')

dim_time  = 'valid_time' if 'valid_time' in ds_era.coords else 'time'
dim_level = 'pressure_level' if 'pressure_level' in ds_era.coords else 'level'

# VAPOR DE AGUA + VORTICIDAD C8, C09 Y C11


In [ ]:

# =========================================================
# 4. ABRIR ERA5
# =========================================================
era = xr.open_dataset(era5_file)

if era.longitude.max() > 180:
    era['longitude'] = (era.longitude + 180) % 360 - 180
    era = era.sortby('longitude')

# =========================================================
# 5. FUNCIÓN VORTICIDAD
# =========================================================
def vorticidad(u, v, lat, lon):
    Re = 6.371e6
    lat_rad = np.deg2rad(lat)

    dy = np.gradient(lat_rad) * Re
    dx = np.gradient(np.deg2rad(lon)) * Re * np.cos(lat_rad[:, None])

    dvdx = np.gradient(v, axis=1) / dx
    dudy = np.gradient(u, axis=0) / dy[:, None]

    return dvdx - dudy

# =========================================================
# 6. LOOP PRINCIPAL
# =========================================================
for target in targets:

    try:

        print(f"Procesando {target}")
        carpeta = os.path.join(ruta_base, target)

        def canal(c):
            return os.path.join(carpeta, [x for x in os.listdir(carpeta) if c in x][0])

        ds8  = GOES.open_dataset(canal("C08"))
        ds9  = GOES.open_dataset(canal("C09"))
        ds11 = GOES.open_dataset(canal("C11"))

        C08, lon, lat = ds8.image("CMI", lonlat="center", domain=domain)
        C09, _, _     = ds9.image("CMI", lonlat="center", domain=domain)
        C11, _, _     = ds11.image("CMI", lonlat="center", domain=domain)

        C08 = C08.data - 273.15
        C09 = C09.data - 273.15
        C11 = C11.data - 273.15

        # ============================
        # RGB Vapor de Agua (C08–C09–C11)
        # ============================

        R = np.clip(C08 - C11, -25, 5)   # Vapor alto – intrusiones secas
        G = np.clip(C09 - C11, -35, 10)  # Vapor medio
        B = np.clip(C11, -70, -20)       # Fondo térmico IR

        Rn = (R + 25) / 30
        Gn = (G + 35) / 45
        Bn = (B + 70) / 50

        RGB = np.clip(np.dstack([Rn, Gn, Bn]), 0, 1)


        # ============================
        # ERA5 500 hPa
        # ============================
        fecha = f"{target[:4]}-{target[4:6]}-{target[6:8]}T{target[9:11]}:00"

        era500 = era.sel(
            pressure_level=500,
            valid_time=fecha,
            method="nearest"
        )

        u = era500.u.values
        v = era500.v.values
        lat_e = era500.latitude.values
        lon_e = era500.longitude.values

        vort = vorticidad(u, v, lat_e, lon_e) * 1e5
        vort = gaussian_filter(vort, sigma=2)

        # ============================
        # PLOT
        # ============================
        fig = plt.figure(figsize=(10,10))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.set_extent(domain)

        ax.imshow(
            RGB,
            origin="upper",
            extent=[domain[0], domain[1], domain[2], domain[3]],
            transform=ccrs.PlateCarree(),
            zorder=1
        )

        levels = np.arange(-10, 11, 2)

        cs = ax.contour(
            lon_e, lat_e, vort,
            levels=levels,
            colors="white",
            linewidths=0.6,
            transform=ccrs.PlateCarree(),
            zorder=3
        )

        ax.clabel(cs, fontsize=7)

        ax.add_feature(cfeature.LAND, facecolor="none", edgecolor="black", linewidth=0.6, zorder=4)
        ax.add_feature(cfeature.COASTLINE, linewidth=1.2, color="black", zorder=5)
        ax.add_feature(cfeature.BORDERS, linewidth=0.8, color="black", zorder=5)

        gl = ax.gridlines(draw_labels=True, linestyle="--", alpha=0.4)
        gl.top_labels = False
        gl.right_labels = False
        gl.xformatter = LongitudeFormatter()
        gl.yformatter = LatitudeFormatter()

        plt.title(f"RGB Vapor Agua + Vorticidad 500 hPa\n{target} UTC")

        salida = os.path.join(output_dir, f"VAPOR_VORT500_{target}.png")
        plt.savefig(salida, dpi=250, bbox_inches="tight")
        plt.close()

        print(f"✅ Guardado: {salida}")

        # ============================
        # LIMPIEZA RAM
        # ============================
        try:
            ds8.close()
            ds9.close()
            ds11.close()
        except:
            pass

        del C08, C09, C11
        del R, G, B, Rn, Gn, Bn, RGB
        del u, v, vort
        del era500

        gc.collect()

    except Exception as e:
        print(f"❌ Error en {target}: {e}")


Procesando 20250624_12
✅ Guardado: /content/drive/MyDrive/FIGURAS_VAPOR_VORT500/VAPOR_VORT500_20250624_12.png
Procesando 20250625_12
✅ Guardado: /content/drive/MyDrive/FIGURAS_VAPOR_VORT500/VAPOR_VORT500_20250625_12.png
Procesando 20250626_12
✅ Guardado: /content/drive/MyDrive/FIGURAS_VAPOR_VORT500/VAPOR_VORT500_20250626_12.png
Procesando 20250627_12
✅ Guardado: /content/drive/MyDrive/FIGURAS_VAPOR_VORT500/VAPOR_VORT500_20250627_12.png
Procesando 20250628_12
✅ Guardado: /content/drive/MyDrive/FIGURAS_VAPOR_VORT500/VAPOR_VORT500_20250628_12.png
Procesando 20250629_12
✅ Guardado: /content/drive/MyDrive/FIGURAS_VAPOR_VORT500/VAPOR_VORT500_20250629_12.png
Procesando 20250630_12
✅ Guardado: /content/drive/MyDrive/FIGURAS_VAPOR_VORT500/VAPOR_VORT500_20250630_12.png


# MASA DE AIRE + VORTICIDAD C8, C11 Y C13


In [ ]:

# =========================================================
# ABRIR ERA5
# =========================================================
era = xr.open_dataset(era5_file)

if era.longitude.max() > 180:
    era['longitude'] = (era.longitude + 180) % 360 - 180
    era = era.sortby('longitude')

# =========================================================
# FUNCIÓN VORTICIDAD
# =========================================================
def vorticidad(u, v, lat, lon):

    Re = 6.371e6
    lat_rad = np.deg2rad(lat)

    dy = np.gradient(lat_rad) * Re
    dx = np.gradient(np.deg2rad(lon)) * Re * np.cos(lat_rad[:, None])

    dvdx = np.gradient(v, axis=1) / dx
    dudy = np.gradient(u, axis=0) / dy[:, None]

    return dvdx - dudy

# =========================================================
# LOOP AIR MASS RGB + VORTICIDAD 500
# =========================================================
for target in targets:

    try:

        print(f"Procesando {target}")

        carpeta = os.path.join(ruta_base, target)

        # ============================
        # FUNCIÓN BUSCAR CANAL
        # ============================
        def canal(c):
            files = [x for x in os.listdir(carpeta) if c in x]
            if len(files) == 0:
                raise ValueError(f"No existe canal {c}")
            return os.path.join(carpeta, files[0])

        # ============================
        # GOES
        # ============================

        ds8  = GOES.open_dataset(canal("C08"))
        ds11 = GOES.open_dataset(canal("C11"))
        ds13 = GOES.open_dataset(canal("C13"))

        C08, lon, lat = ds8.image("CMI", lonlat="center", domain=domain)
        C11, _, _     = ds11.image("CMI", lonlat="center", domain=domain)
        C13, _, _     = ds13.image("CMI", lonlat="center", domain=domain)

        # Convertir a °C
        C08 = C08.data - 273.15
        C11 = C11.data - 273.15
        C13 = C13.data - 273.15

        # =================================================
        # RGB MASAS DE AIRE


        # Diferencias espectrales (CLAVE)
        R = np.clip(C08 - C13, -30, 5)   # Aire seco en altura
        G = np.clip(C11 - C13, -35, 10)  # Vapor medio
        B = np.clip(C13, -80, -20)       # Aire frío / nubes altas

        # Normalización
        Rn = (R + 30) / 35
        Gn = (G + 35) / 45
        Bn = (B + 80) / 60

        RGB = np.clip(np.dstack([Rn, Gn, Bn]), 0, 1)


        # =================================================
        # ERA5 500 hPa
        # =================================================
        fecha = f"{target[:4]}-{target[4:6]}-{target[6:8]}T{target[9:11]}:00"

        era500 = era.sel(
            pressure_level=500,
            valid_time=fecha,
            method="nearest"
        )

        u = era500.u.values
        v = era500.v.values
        lat_e = era500.latitude.values
        lon_e = era500.longitude.values

        vort = vorticidad(u, v, lat_e, lon_e) * 1e5
        vort = gaussian_filter(vort, sigma=2)

        # =================================================
        # PLOT
        # =================================================
        fig = plt.figure(figsize=(10,10))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.set_extent(domain)

        ax.imshow(
            RGB,
            origin="upper",
            extent=[domain[0], domain[1], domain[2], domain[3]],
            transform=ccrs.PlateCarree(),
            zorder=1
        )

        levels = np.arange(-12, 13, 2)

        cs = ax.contour(
            lon_e, lat_e, vort,
            levels=levels,
            colors="gray",
            linewidths=0.6,
            transform=ccrs.PlateCarree(),
            zorder=3
        )

        ax.clabel(cs, fontsize=7)

        ax.add_feature(cfeature.LAND, facecolor="none", edgecolor="black", linewidth=0.6, zorder=4)
        ax.add_feature(cfeature.COASTLINE, linewidth=1.2, color="black", zorder=5)
        ax.add_feature(cfeature.BORDERS, linewidth=0.8, color="black", zorder=5)

        gl = ax.gridlines(draw_labels=True, linestyle="--", alpha=0.4)
        gl.top_labels = False
        gl.right_labels = False
        gl.xformatter = LongitudeFormatter()
        gl.yformatter = LatitudeFormatter()

        plt.title(f"Air Mass RGB + Vorticidad 500 hPa\n{target} UTC")

        salida = os.path.join(output_dir, f"AIRMASS_VORT500ofii_{target}.png")
        plt.savefig(salida, dpi=250, bbox_inches="tight")
        plt.close()

        print(f"✅ Guardado: {salida}")

        # =================================================
        # LIMPIEZA RAM
        # =================================================
        try:
            ds8.close()
            ds9.close()
            ds11.close()
        except:
            pass

        del C08, C11, C13
        del R, G, B, Rn, Gn, Bn, RGB
        del u, v, vort
        del era500

        gc.collect()

    except Exception as e:
        print(f"❌ Error en {target}: {e}")


Procesando 20250624_12
✅ Guardado: /content/drive/MyDrive/FIGURAS_VAPOR_VORT500/AIRMASS_VORT500ofii_20250624_12.png
Procesando 20250625_12
✅ Guardado: /content/drive/MyDrive/FIGURAS_VAPOR_VORT500/AIRMASS_VORT500ofii_20250625_12.png
Procesando 20250626_12
✅ Guardado: /content/drive/MyDrive/FIGURAS_VAPOR_VORT500/AIRMASS_VORT500ofii_20250626_12.png
Procesando 20250627_12
✅ Guardado: /content/drive/MyDrive/FIGURAS_VAPOR_VORT500/AIRMASS_VORT500ofii_20250627_12.png
Procesando 20250628_12
❌ Error en 20250628_12: No existe canal C08
Procesando 20250629_12
✅ Guardado: /content/drive/MyDrive/FIGURAS_VAPOR_VORT500/AIRMASS_VORT500ofii_20250629_12.png
Procesando 20250630_12
✅ Guardado: /content/drive/MyDrive/FIGURAS_VAPOR_VORT500/AIRMASS_VORT500ofii_20250630_12.png


#PLOTEO DEL GIFF

In [ ]:
!pip install imageio


In [ ]:
import imageio.v2 as imageio
import glob
import os

ruta_imagenes = "/content/drive/MyDrive/FIGURAS_VAPOR_VORT500"
ruta_salida_gif = "/content/drive/MyDrive/GOES19_ATACAMA_20250624_20250630/DEVERITAS/GIF_ATACAMA_RGB.gif"

# Buscar todas las imágenes PNG
files = sorted(glob.glob(ruta_imagenes + "VAPOR_VORT500_*.png"))

print("Imágenes encontradas:", len(files))

# Crear GIF (modo RAM bajo)
with imageio.get_writer(ruta_salida_gif, mode="I", duration=1.5) as writer:
    for filename in files:
        print("Agregando:", os.path.basename(filename))
        image = imageio.imread(filename)
        writer.append_data(image)

print("GIF creado en:", ruta_salida_gif)